In [1]:
import pandas as pd
import numpy as np

from scipy import stats

from sklearn import metrics
from sklearn import linear_model
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split
import pickle

from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from statsmodels.formula.api import ols
pd.set_option('display.max_columns', 300)


## Step 1: Read in hold out data, scalers, and best model

In [2]:
df = pd.read_csv('kc_house_data_test_features.csv', index_col=0)

In [3]:
pickle_in = open("scaler.pickle","rb")
final_scaler = pickle.load(pickle_in)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.preprocessing.data module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.21.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [4]:
pickle_in = open("prediction.pickle","rb")
final_model = pickle.load(pickle_in)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values and scaling) that we performed on the original data.  

In [5]:
# transformed_holdout = final_scaler(holdout)

## Extreme Values

In [6]:
df2 = df[(df['bedrooms']<=6)|(df['bathrooms']<=5)|(df['sqft_living']<=5000)|(df['sqft_lot']<=135000)|(df['floors']<=3)|(df['sqft_above']<=5500)|(df['sqft_basement']<=1500)|(df['sqft_living15']<=4500)|(df['sqft_lot15']<=100000)]


In [7]:
features = ['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
            'sqft_living15', 'sqft_lot15']

df_features = df2[features]


In [8]:
poly_2 = PolynomialFeatures(degree=2, include_bias=False)
poly2_data = poly_2.fit_transform(df_features)
poly2_columns = poly_2.get_feature_names(df_features.columns)
df_poly2 = pd.DataFrame(poly2_data, columns=poly2_columns)

## Step 3: Predict the holdout set

In [9]:
# final_answers = final_model.predict(transformed_holdout)

In [10]:
df_poly2 = pd.DataFrame(data=final_scaler.transform(df_poly2), columns=df_poly2.columns)

In [11]:
results = final_model.predict(df_poly2)

## Step 4: Export your predictions

In [12]:
final_predictions = pd.Series(data = results)

In [13]:
final_predictions

0       547067.555896
1       547067.555896
2       374546.305896
3       364255.555896
4       537754.555896
            ...      
4318    483951.555896
4319    475247.555896
4320    324855.305896
4321    384694.305896
4322    327522.055896
Length: 4323, dtype: float64

In [14]:
final_predictions.to_csv('housing_preds_DCJV.csv')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
